In [9]:
import io
import pandas as pd
import re
import numpy as np

data = pd.read_csv('movie_data_new.csv', header=None, names=['New Book Name','ID','Link','Budget','Gross','Opening','Release Date'], sep='\t')

data['Link'] = data['Link'].map(lambda x: re.search("(.*?)business", x).group(1).strip())

book_data = pd.read_csv('goodreads_data_modified.csv', encoding='utf-8')

book_data = book_data.drop('Unnamed: 0', axis=1)

book_data = book_data.drop_duplicates()
data = data.drop_duplicates()

In [10]:
Merged = pd.merge(data, book_data, how='left', on='New Book Name', suffixes=('_x', '_y'))

In [11]:
Merged = Merged.drop_duplicates()
Merged = Merged.set_index([range(Merged['Link'].values.size)])

Merged['New Book Name'].values.size

1214

In [12]:
Merged = Merged.groupby('New Book Name', as_index=False).max()

Merged['New Book Name'].values.size

1214

In [13]:
import datetime
from datetime import datetime
from dateutil import parser

def convert_to_datetime_publishdate(x):
    if(x!='None'): 
        try:
            arr = x.split(' ')
            x = arr[0].strip()+' '+re.search('(.*?)\D', arr[1].strip()).group(1)+' '+arr[2].strip()
            #print type(x)
            #datetime_obj = datetime.strftime(x , '%B %d %Y')
            #print type(datetime_obj)
            datetime_obj = parser.parse(x)
            return datetime_obj
        except Exception:
            try:
                arr = x.split(' ')
                x = arr[0].strip()+' '+arr[1].strip()
                #print x
                #datetime_obj = datetime.strptime(x, '%B %Y')
                datetime_obj = parser.parse(x)
                return datetime_obj
            except Exception:
                try:
                    arr = x.split('-')
                    x = arr[0].strip()+' '+arr[1].strip()
                    #print x
                    #datetime_obj = datetime.strptime(x, '%b %y')
                    datetime_obj = parser.parse(x)
                    return datetime_obj
                except Exception:
                    try:
                        x = x.strip()
                        #print x
                        #datetime_obj = datetime.strptime(x, '%Y')
                        datetime_obj = parser.parse(x)
                        return datetime_obj
                    except Exception:
                        x = x.strip()
                        x = '0'+x
                        #print x
                        #datetime_obj = datetime.strptime(x, '%Y')
                        datetime_obj = parser.parse(x)
                        return datetime_obj
    else:
        return np.datetime64('NaT')
    
Merged['publish_date'] = Merged['publish_date'].fillna(np.datetime64('NaT'))
Merged['publish_date'] = Merged['publish_date'].astype('str').map(convert_to_datetime_publishdate)

Merged['publish_date']= pd.to_datetime(Merged['publish_date'], coerce=True)


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:51: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead


In [14]:
def convert_to_datetime_releasedate(x):
    if(x!='nan'):
        try:
            arr = x.split(' ')
            x = arr[0].strip()+' '+arr[1].strip()+' '+arr[2].strip()
            datetime_obj = datetime.strptime(x , '%d %B %Y')
            return datetime_obj
        except IndexError:
            try:
                arr = x.split(' ')
                x = arr[0].strip()+' '+arr[1].strip()
                datetime_obj = datetime.strptime(x, '%B %Y')
                return datetime_obj
            except IndexError:
                x = x.strip()
                datetime_obj = datetime.strptime(x, '%Y')
                return datetime_obj
    else:
        return x

Merged['Release Date'] = Merged['Release Date'].astype('str').map(convert_to_datetime_releasedate)

In [15]:
Merged['Time Diff'] = Merged['Release Date']-Merged['publish_date']

In [16]:
Merged['Time Diff']
Merged.to_csv('Merged_Dataset_new.csv', encoding='utf-8')